In [1]:
import cv2
cv2.__version__

sift = cv2.xfeatures2d.SIFT_create()
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create()

In [95]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook, tqdm
path = './BoWdata'
codebook_len = 576
img_size = 288
jump = 6

In [16]:
## 라벨링 정보를 불러와 사전형으로 저장
label_pd = pd.read_csv(path + '/Label2Names.csv',header = None)
label_data = np.array(label_pd)

label_dict = {}
for x in range(len(label_data)):
    label_dict[label_data[x][1]] = label_data[x][0]
label_dict['BACKGROUND_Google'] = 102

In [20]:
## 이미지를 불러오며 해당 이미지의 라벨과 기술정보를 저장
path_train = path + '/train/'
all_train_des = []
all_train_label = []
a = 0
for label in os.listdir(path_train):
    print(path_train+label,label_dict[label])
    for file in os.listdir(path_train+label):
        img = cv2.imread(path_train+label+'/'+file,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(img_size,img_size))    
        kp = [cv2.KeyPoint(x,y,jump) for x in range(0,img.shape[1],jump) for y in range(0,img.shape[0],jump)]    
        _,des =sift.compute(img,kp)
        all_train_des.append(des)
        all_train_label.append(label_dict[label])
    a+=1
    print(int((a/len(os.listdir(path_train)))*100+0.5),'%')

./data/train/flamingo 42
1 %
./data/train/platypus 74
2 %
./data/train/crocodile_head 30
3 %
./data/train/hawksbill 48
4 %
./data/train/sunflower 91
5 %
./data/train/kangaroo 55
6 %
./data/train/schooner 80
7 %
./data/train/ceiling_fan 21
8 %
./data/train/watch 95
9 %
./data/train/elephant 37
10 %
./data/train/lobster 60
11 %
./data/train/soccer_ball 85
12 %
./data/train/llama 59
13 %
./data/train/scorpion 82
14 %
./data/train/anchor 7
15 %
./data/train/pyramid 75
16 %
./data/train/lotus 61
17 %
./data/train/scissors 81
18 %
./data/train/cup 31
19 %
./data/train/minaret 66
20 %
./data/train/cellphone 22
21 %
./data/train/buddha 16
22 %
./data/train/beaver 11
23 %
./data/train/sea_horse 83
24 %
./data/train/yin_yang 101
25 %
./data/train/saxophone 79
25 %
./data/train/flamingo_head 43
26 %
./data/train/rhino 77
27 %
./data/train/okapi 69
28 %
./data/train/Faces 1
29 %
./data/train/water_lilly 96
30 %
./data/train/binocular 12
31 %
./data/train/pigeon 72
32 %
./data/train/barrel 9
33 %
.

In [21]:
np.save(path+'/save/All_des_288x288_6.npy',all_train_des)

In [23]:
np.save(path+'/save/All_label.npy',all_train_label)

In [23]:
all_train_des = np.load(path+'/test_des_288x288_6.npy')

FileNotFoundError: ignored

In [ ]:
all_train_label = np.load(path+'/All_label.npy')

In [24]:
train_des = []
train_label = []
check_des = []
check_label = []
for x in range(102):
    for y in range(30):
        if y < 25 :
            train_des.append(all_train_des[x*30+y])
            train_label.append(all_train_label[x*30+y])
        else :
            check_des.append(all_train_des[x*30+y])
            check_label.append(all_train_label[x*30+y])
print(np.array(train_des).shape,np.array(train_label).shape)
print(np.array(check_des).shape,np.array(check_label).shape)

(2550, 2304, 128) (2550,)
(510, 2304, 128) (510,)


In [ ]:
## 학습 이미지의 기술자정보 합침
des_list= []
for x in range(len(train_des)) :
    des_list.extend(train_des[x])

In [31]:
import kmc2
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

In [ ]:
#코드북 생성
seeding = kmc2.kmc2(des_list,codebook_len)
clustered_mini = MiniBatchKMeans(codebook_len,init=seeding,n_init=1,init_size=int(codebook_len*3.3)).fit(des_list)
codebook = clustered_mini.cluster_centers_

In [25]:
from scipy.cluster.vq import vq

In [ ]:
## 각각의 이미지의 코드북에대한 히스토그램
def SPM (des,codebook,codebook_len,img_size,step):

  cut1 = int((img_size/step)*(1/4))
  cut2 = int((img_size/step)*(2/4))
  cut3 = int((img_size/step)*(3/4))
  histogram = []
  histograma = []

  for x in range(len(des)):
    des_map = des[x].reshape(int(img_size/step),int(img_size/step),128)

    ## level 0
    lv_0,_ = vq(des[x],codebook)
    #
    lv_0,_ = np.histogram(lv_0, bins=list(range(codebook_len+1)),normed=True)


    #
    ## level 1
    lv_1_1,_ = vq(des_map[:cut2,:cut2,:].reshape(-1,128),codebook)
    lv_1_1a,_ = np.histogram(lv_1_1, bins=list(range(codebook_len+1)),normed=True)
    lv_1_1a = lv_1_1a.reshape((codebook_len,1))

    lv_1_2,_ = vq(des_map[cut2:,:cut2,:].reshape(-1,128),codebook)
    lv_1_2a,_ = np.histogram(lv_1_2, bins=list(range(codebook_len+1)),normed=True)
    lv_1_2a = lv_1_2a.reshape((codebook_len,1))

    lv_1_3,_ = vq(des_map[:cut2,cut2:,:].reshape(-1,128),codebook)
    lv_1_3a,_ = np.histogram(lv_1_3, bins=list(range(codebook_len+1)),normed=True)
    lv_1_3a = lv_1_3a.reshape((codebook_len,1))

    lv_1_4,_ = vq(des_map[cut2:,cut2:,:].reshape(-1,128),codebook)
    lv_1_4a,_ = np.histogram(lv_1_4, bins=list(range(codebook_len+1)),normed=True)
    lv_1_4a = lv_1_4a.reshape((codebook_len,1))

  #
    lv_1 = np.concatenate((lv_1_1, lv_1_2, lv_1_3, lv_1_4))
    lv_1,_ = np.histogram(lv_1, bins=list(range(codebook_len+1)),normed=True)

    lv_1a = np.concatenate((lv_1_1a,lv_1_2a,lv_1_3a,lv_1_4a),axis=1).flatten()

    
    #
    ## level 2
    lv_2_1,_ = vq(des_map[:cut1,:cut1,:].reshape(-1,128),codebook)
    lv_2_1a,_ = np.histogram(lv_2_1,bins=list(range(codebook_len+1)),normed=True)
    lv_2_1a = lv_2_1a.reshape((codebook_len,1))
    lv_2_2,_ = vq(des_map[cut1:,:cut1,:].reshape(-1,128),codebook)
    lv_2_2a,_ = np.histogram(lv_2_2,bins=list(range(codebook_len+1)),normed=True)
    lv_2_2a = lv_2_2a.reshape((codebook_len,1))
    lv_2_3,_ = vq(des_map[cut2:cut3,:cut1,:].reshape(-1,128),codebook)
    lv_2_3a,_ = np.histogram(lv_2_3,bins=list(range(codebook_len+1)),normed=True)
    lv_2_3a = lv_2_3a.reshape((codebook_len,1))
    lv_2_4,_ = vq(des_map[cut3:,:cut1,:].reshape(-1,128),codebook)
    lv_2_4a,_ = np.histogram(lv_2_4,bins=list(range(codebook_len+1)),normed=True)
    lv_2_4a = lv_2_4a.reshape((codebook_len,1))
    #
    lv_2_5,_ = vq(des_map[:cut1,cut1:cut2,:].reshape(-1,128),codebook)
    lv_2_5a,_ = np.histogram(lv_2_5,bins=list(range(codebook_len+1)),normed=True)
    lv_2_5a = lv_2_5a.reshape((codebook_len,1))
    lv_2_6,_ = vq(des_map[cut1:,cut1:cut2,:].reshape(-1,128),codebook)
    lv_2_6a,_ = np.histogram(lv_2_6,bins=list(range(codebook_len+1)),normed=True)
    lv_2_6a = lv_2_6a.reshape((codebook_len,1))
    lv_2_7,_ = vq(des_map[cut2:cut3,cut1:cut2,:].reshape(-1,128),codebook)
    lv_2_7a,_ = np.histogram(lv_2_7,bins=list(range(codebook_len+1)),normed=True)
    lv_2_7a = lv_2_7a.reshape((codebook_len,1))
    lv_2_8,_ = vq(des_map[cut3:,cut1:cut2,:].reshape(-1,128),codebook)
    lv_2_8a,_ = np.histogram(lv_2_8,bins=list(range(codebook_len+1)),normed=True)
    lv_2_8a = lv_2_8a.reshape((codebook_len,1))
    #
    lv_2_9,_ = vq(des_map[:cut1,cut2:cut3,:].reshape(-1,128),codebook)
    lv_2_9a,_ = np.histogram(lv_2_9,bins=list(range(codebook_len+1)),normed=True)
    lv_2_9a = lv_2_9a.reshape((codebook_len,1))
    lv_2_10,_ = vq(des_map[cut1:,cut2:cut3,:].reshape(-1,128),codebook)
    lv_2_10a,_ = np.histogram(lv_2_10,bins=list(range(codebook_len+1)),normed=True)
    lv_2_10a = lv_2_10a.reshape((codebook_len,1))
    lv_2_11,_ = vq(des_map[cut2:cut3,cut2:cut3,:].reshape(-1,128),codebook)
    lv_2_11a,_ = np.histogram(lv_2_11,bins=list(range(codebook_len+1)),normed=True)
    lv_2_11a = lv_2_11a.reshape((codebook_len,1))
    lv_2_12,_ = vq(des_map[cut3:,cut2:cut3,:].reshape(-1,128),codebook)
    lv_2_12a,_ = np.histogram(lv_2_12,bins=list(range(codebook_len+1)),normed=True)
    lv_2_12a = lv_2_12a.reshape((codebook_len,1))
    #
    lv_2_13,_ = vq(des_map[:cut1,cut3:,:].reshape(-1,128),codebook)
    lv_2_13a,_ = np.histogram(lv_2_13,bins=list(range(codebook_len+1)),normed=True)
    lv_2_13a = lv_2_13a.reshape((codebook_len,1))
    lv_2_14,_ = vq(des_map[cut1:,cut3:,:].reshape(-1,128),codebook)
    lv_2_14a,_ = np.histogram(lv_2_14,bins=list(range(codebook_len+1)),normed=True)
    lv_2_14a = lv_2_14a.reshape((codebook_len,1))
    lv_2_15,_ = vq(des_map[cut2:cut3,cut3:,:].reshape(-1,128),codebook)
    lv_2_15a,_ = np.histogram(lv_2_15,bins=list(range(codebook_len+1)),normed=True)
    lv_2_15a = lv_2_15a.reshape((codebook_len,1))
    lv_2_16,_ = vq(des_map[cut3:,cut3:,:].reshape(-1,128),codebook)
    lv_2_16a,_ = np.histogram(lv_2_16,bins=list(range(codebook_len+1)),normed=True)
    lv_2_16a = lv_2_16a.reshape((codebook_len,1))
    #
    lv_2 = np.concatenate((lv_2_1, lv_2_2, lv_2_3, lv_2_4, lv_2_5, lv_2_6,lv_2_7, lv_2_8,
                          lv_2_9, lv_2_10, lv_2_11, lv_2_12, lv_2_13, lv_2_14, lv_2_15, lv_2_16))
    lv_2,_ = np.histogram(lv_2, bins=list(range(codebook_len+1)),normed=True)

    lv_2a = np.concatenate((lv_2_1a, lv_2_2a, lv_2_3a, lv_2_4a, lv_2_5a, lv_2_6a,lv_2_7a, lv_2_8a,
                          lv_2_9a, lv_2_10a, lv_2_11a, lv_2_12a, lv_2_13a, lv_2_14a, lv_2_15a, lv_2_16a),axis=1).flatten()
    
    hist = lv_0*(0.25)+ lv_1*(0.25)+ lv_2*(0.5)
    hista = np.concatenate((lv_0*(25), lv_1a*(25), lv_2a*(50)))
    histogram.append(hist)
    histograma.append(hista)
     
  return (histograma)

In [71]:
## 각각의 이미지의 코드북에대한 히스토그램
def SPM_2 (des,codebook,codebook_len,img_size,step):
    
    cut1 = int((img_size/step)*(1/4))
    cut2 = int((img_size/step)*(2/4))
    cut3 = int((img_size/step)*(3/4))
    cut4 = int((img_size/step)*(1/3))
    cut5 = int((img_size/step)*(2/3))

    histogram = []    
    
    a = 0
    for x in tqdm(range(len(des))):
        des_map = des[x].reshape(int(img_size/step),int(img_size/step),128)

        ## level 0
        lv_0,_ = vq(des[x],codebook)
        #
        lv_0,_ = np.histogram(lv_0, bins=list(range(codebook_len+1)),normed=True)


        #
        ## level 1
        lv_1_1,_ = vq(des_map[:cut2,:cut2,:].reshape(-1,128),codebook)
        lv_1_1,_ = np.histogram(lv_1_1, bins=list(range(codebook_len+1)),normed=True)
        lv_1_1 = lv_1_1.reshape((codebook_len,1))

        lv_1_2,_ = vq(des_map[cut2:,:cut2,:].reshape(-1,128),codebook)
        lv_1_2,_ = np.histogram(lv_1_2, bins=list(range(codebook_len+1)),normed=True)
        lv_1_2 = lv_1_2.reshape((codebook_len,1))

        lv_1_3,_ = vq(des_map[:cut2,cut2:,:].reshape(-1,128),codebook)
        lv_1_3,_ = np.histogram(lv_1_3, bins=list(range(codebook_len+1)),normed=True)
        lv_1_3 = lv_1_3.reshape((codebook_len,1))

        lv_1_4,_ = vq(des_map[cut2:,cut2:,:].reshape(-1,128),codebook)
        lv_1_4,_ = np.histogram(lv_1_4, bins=list(range(codebook_len+1)),normed=True)
        lv_1_4 = lv_1_4.reshape((codebook_len,1))
        #

        lv_1 = np.concatenate((lv_1_1, lv_1_2, lv_1_3, lv_1_4), axis=1).flatten()

        #level 1.5
        lv_3_1,_ = vq(des_map[:cut4,:cut4,:].reshape(-1,128),codebook)
        lv_3_1,_ = np.histogram(lv_3_1, bins=list(range(codebook_len+1)),normed=True)
        lv_3_1 = lv_3_1.reshape((codebook_len,1))

        lv_3_2,_ = vq(des_map[cut4:cut5,:cut4,:].reshape(-1,128),codebook)
        lv_3_2,_ = np.histogram(lv_3_2, bins=list(range(codebook_len+1)),normed=True)
        lv_3_2 = lv_3_2.reshape((codebook_len,1))

        lv_3_3,_ = vq(des_map[cut5:,:cut4,:].reshape(-1,128),codebook)
        lv_3_3,_ = np.histogram(lv_3_3, bins=list(range(codebook_len+1)),normed=True)
        lv_3_3 = lv_3_3.reshape((codebook_len,1))

        lv_3_4,_ = vq(des_map[:cut4,cut4:cut5,:].reshape(-1,128),codebook)
        lv_3_4,_ = np.histogram(lv_3_4, bins=list(range(codebook_len+1)),normed=True)
        lv_3_4 = lv_3_4.reshape((codebook_len,1))
    
        lv_3_5,_ = vq(des_map[cut4:cut5,cut4:cut5,:].reshape(-1,128),codebook)
        lv_3_5,_ = np.histogram(lv_3_5, bins=list(range(codebook_len+1)),normed=True)
        lv_3_5 = lv_3_5.reshape((codebook_len,1))

        lv_3_6,_ = vq(des_map[cut5:,cut4:cut5,:].reshape(-1,128),codebook)
        lv_3_6,_ = np.histogram(lv_3_6, bins=list(range(codebook_len+1)),normed=True)
        lv_3_6 = lv_3_6.reshape((codebook_len,1))

        lv_3_7,_ = vq(des_map[:cut4,cut5:,:].reshape(-1,128),codebook)
        lv_3_7,_ = np.histogram(lv_3_7, bins=list(range(codebook_len+1)),normed=True)
        lv_3_7 = lv_3_7.reshape((codebook_len,1))

        lv_3_8,_ = vq(des_map[cut4:cut5,cut5:,:].reshape(-1,128),codebook)
        lv_3_8,_ = np.histogram(lv_3_8, bins=list(range(codebook_len+1)),normed=True)
        lv_3_8 = lv_3_8.reshape((codebook_len,1))

        lv_3_9,_ = vq(des_map[cut5:,cut5:,:].reshape(-1,128),codebook)
        lv_3_9,_ = np.histogram(lv_3_9, bins=list(range(codebook_len+1)),normed=True)
        lv_3_9 = lv_3_9.reshape((codebook_len,1))
        
        lv_3 = np.concatenate((lv_3_1, lv_3_2, lv_3_3, lv_3_4, lv_3_5, lv_3_6, lv_3_7, lv_3_8, lv_3_9), axis=1).flatten()
    
        
        ## level 2
        lv_2_1,_ = vq(des_map[:cut1, :cut1, :].reshape(-1,128),codebook)
        lv_2_1,_ = np.histogram(lv_2_1,bins=list(range(codebook_len+1)),normed=True)
        lv_2_1 = lv_2_1.reshape((codebook_len,1))
        lv_2_2,_ = vq(des_map[cut1:, :cut1, :].reshape(-1,128),codebook)
        lv_2_2,_ = np.histogram(lv_2_2,bins=list(range(codebook_len+1)),normed=True)
        lv_2_2 = lv_2_2.reshape((codebook_len,1))
        lv_2_3,_ = vq(des_map[cut2:cut3, :cut1, :].reshape(-1,128),codebook)
        lv_2_3,_ = np.histogram(lv_2_3,bins=list(range(codebook_len+1)),normed=True)
        lv_2_3 = lv_2_3.reshape((codebook_len,1))
        lv_2_4,_ = vq(des_map[cut3:,:cut1,:].reshape(-1,128),codebook)
        lv_2_4,_ = np.histogram(lv_2_4,bins=list(range(codebook_len+1)),normed=True)
        lv_2_4 = lv_2_4.reshape((codebook_len,1))
        #
        lv_2_5,_ = vq(des_map[:cut1,cut1:cut2,:].reshape(-1,128),codebook)
        lv_2_5,_ = np.histogram(lv_2_5,bins=list(range(codebook_len+1)),normed=True)
        lv_2_5 = lv_2_5.reshape((codebook_len,1))
        lv_2_6,_ = vq(des_map[cut1:,cut1:cut2,:].reshape(-1,128),codebook)
        lv_2_6,_ = np.histogram(lv_2_6,bins=list(range(codebook_len+1)),normed=True)
        lv_2_6 = lv_2_6.reshape((codebook_len,1))
        lv_2_7,_ = vq(des_map[cut2:cut3,cut1:cut2,:].reshape(-1,128),codebook)
        lv_2_7,_ = np.histogram(lv_2_7,bins=list(range(codebook_len+1)),normed=True)
        lv_2_7 = lv_2_7.reshape((codebook_len,1))
        lv_2_8,_ = vq(des_map[cut3:,cut1:cut2,:].reshape(-1,128),codebook)
        lv_2_8,_ = np.histogram(lv_2_8,bins=list(range(codebook_len+1)),normed=True)
        lv_2_8 = lv_2_8.reshape((codebook_len,1))
        #
        lv_2_9,_ = vq(des_map[:cut1,cut2:cut3,:].reshape(-1,128),codebook)
        lv_2_9,_ = np.histogram(lv_2_9,bins=list(range(codebook_len+1)),normed=True)
        lv_2_9 = lv_2_9.reshape((codebook_len,1))
        lv_2_10,_ = vq(des_map[cut1:,cut2:cut3,:].reshape(-1,128),codebook)
        lv_2_10,_ = np.histogram(lv_2_10,bins=list(range(codebook_len+1)),normed=True)
        lv_2_10 = lv_2_10.reshape((codebook_len,1))
        lv_2_11,_ = vq(des_map[cut2:cut3,cut2:cut3,:].reshape(-1,128),codebook)
        lv_2_11,_ = np.histogram(lv_2_11,bins=list(range(codebook_len+1)),normed=True)
        lv_2_11 = lv_2_11.reshape((codebook_len,1))
        lv_2_12,_ = vq(des_map[cut3:,cut2:cut3,:].reshape(-1,128),codebook)
        lv_2_12,_ = np.histogram(lv_2_12,bins=list(range(codebook_len+1)),normed=True)
        lv_2_12 = lv_2_12.reshape((codebook_len,1))
        #
        lv_2_13,_ = vq(des_map[:cut1,cut3:,:].reshape(-1,128),codebook)
        lv_2_13,_ = np.histogram(lv_2_13,bins=list(range(codebook_len+1)),normed=True)
        lv_2_13 = lv_2_13.reshape((codebook_len,1))
        lv_2_14,_ = vq(des_map[cut1:,cut3:,:].reshape(-1,128),codebook)
        lv_2_14,_ = np.histogram(lv_2_14,bins=list(range(codebook_len+1)),normed=True)
        lv_2_14 = lv_2_14.reshape((codebook_len,1))
        lv_2_15,_ = vq(des_map[cut2:cut3,cut3:,:].reshape(-1,128),codebook)
        lv_2_15,_ = np.histogram(lv_2_15,bins=list(range(codebook_len+1)),normed=True)
        lv_2_15 = lv_2_15.reshape((codebook_len,1))
        lv_2_16,_ = vq(des_map[cut3:,cut3:,:].reshape(-1,128),codebook)
        lv_2_16,_ = np.histogram(lv_2_16,bins=list(range(codebook_len+1)),normed=True)
        lv_2_16 = lv_2_16.reshape((codebook_len,1))
        #
    
        lv_2 = np.concatenate((lv_2_1, lv_2_2, lv_2_3, lv_2_4, lv_2_5, lv_2_6,lv_2_7, lv_2_8,
                          lv_2_9, lv_2_10, lv_2_11, lv_2_12, lv_2_13, lv_2_14, lv_2_15, lv_2_16), axis=1).flatten()
    
    
        hist = np.concatenate((lv_0*(15), lv_1*(20), lv_3*(25), lv_2*(40)))
        histogram.append(hist)
        a+=1
        print(int((a/len(des))*100+0.5),'%')
     
    return (histogram)

In [72]:
def HistogramIntersection(M, N):
    m = np.array(M).shape[0]
    n = np.array(N).shape[0]
    result = np.zeros((m,n))
    for i in tqdm(range(m)):
        for j in tqdm(range(n)):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result

In [ ]:
## 학습 데이터 가공 SPM, HistotramIntersection
train_histogram = SPM(train_des,codebook,codebook_len,img_size,jump)
train_histogram = HistogramIntersection(train_histogram,train_histogram)

In [28]:
from sklearn import svm

In [ ]:
## 모델 학습
model = svm.SVC(kernel='precomputed').fit(train_histogram, train_label)

In [ ]:
## 확인 데이터 가공
check_histogram = SPM(check_des,codebook,codebook_len,img_size,jump)
check_histogram = HistogramIntersection(histogram_check,train_histogram)

In [ ]:
## 결과 예측
predict_check = model.predict(check_histogram)

In [ ]:
## 예측 점수 출력
score = 0
for x in range(len(predict_check)):
  if (predict_check[x] == check_label[x]):
    score = score + 1  
print('%.2f%%'%((score/len(predict_check))*100))

#no spm 33.73
#spm 42.94 40.39
#HI 256x256_8 53.33%
#HI 288x288_6 52.55%

In [29]:
## 모든 이미지의 기술정보 합침
all_des_list= []
for x in range(len(all_train_des)) :
    all_des_list.extend(all_train_des[x])

In [32]:
#코드북 생성
all_seeding = kmc2.kmc2(all_des_list,codebook_len)
print("seeding done")
all_clustered_mini = MiniBatchKMeans(codebook_len,init=all_seeding,n_init=1,init_size=int(codebook_len*3.3)).fit(all_des_list)
all_codebook = all_clustered_mini.cluster_centers_

In [ ]:
np.save(path+'/all_codebook_576_.npy',all_codebook)

In [ ]:
all_codebook = all_clustered_mini.cluster_centers_

In [39]:
## 모든 이미지로 학습
## 학습 데이터 가공
all_train_histogram = SPM_2(all_train_des, all_codebook, codebook_len, img_size, jump)
print("SPM done")
all_train_histogram_HI = HistogramIntersection(all_train_histogram, all_train_histogram)

/home/jslee/.conda/envs/bow/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
/home/jslee/.conda/envs/bow/lib/python3.6/site-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
/home/jslee/.conda/envs/bow/lib/python3.6/site-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Passing `normed=True` on non-un

In [40]:
model_all = svm.SVC(kernel="precomputed").fit(all_train_histogram_HI, all_train_label)

In [50]:
os.listdir(path+'/test/')

['img_01441.jpg',
 'img_01337.jpg',
 'img_01085.jpg',
 'img_00233.jpg',
 'img_00823.jpg',
 'img_01412.jpg',
 'img_00009.jpg',
 'img_01175.jpg',
 'img_00912.jpg',
 'img_01503.jpg',
 'img_00308.jpg',
 'img_01709.jpg',
 'img_00932.jpg',
 'img_01563.jpg',
 'img_00741.jpg',
 'img_01029.jpg',
 'img_01518.jpg',
 'img_00846.jpg',
 'img_00479.jpg',
 'img_00367.jpg',
 'img_01474.jpg',
 'img_01549.jpg',
 'img_00471.jpg',
 'img_00544.jpg',
 'img_00635.jpg',
 'img_00560.jpg',
 'img_00290.jpg',
 'img_00160.jpg',
 'img_01540.jpg',
 'img_00084.jpg',
 'img_00123.jpg',
 'img_00180.jpg',
 'img_01132.jpg',
 'img_00613.jpg',
 'img_00833.jpg',
 'img_01065.jpg',
 'img_00222.jpg',
 'img_00355.jpg',
 'img_01201.jpg',
 'img_00289.jpg',
 'img_01280.jpg',
 'img_01300.jpg',
 'img_01126.jpg',
 'img_01204.jpg',
 'img_00162.jpg',
 'img_00044.jpg',
 'img_00595.jpg',
 'img_00727.jpg',
 'img_01463.jpg',
 'img_00244.jpg',
 'img_00557.jpg',
 'img_00047.jpg',
 'img_00306.jpg',
 'img_00209.jpg',
 'img_00641.jpg',
 'img_0169

In [96]:
# 테스트 이미지를 불러오며 각 이미지의 이름과 기술정보를 저장
path_test = path + '/test/'
#test_des = []
test_file_name = []
for file in tqdm(os.listdir(path_test)):
    if(file[0] !='i'):
        print(file)
        continue
    #img = cv2.imread(path_test+file,cv2.IMREAD_GRAYSCALE)
    #img = cv2.resize(img,(img_size,img_size))
    #kp = [cv2.KeyPoint(x,y,jump) for x in range(0,img.shape[1],jump) for y in range(0,img.shape[0],jump)] 
    #_,des =sift.compute(img,kp)
    #test_des.append(des)
    test_file_name.append(file)
    







100%|██████████| 1713/1713 [00:00<00:00, 1461819.48it/s][A

.ipynb_checkpoints


In [92]:
for file in os.listdir(path_test):
    print(file)

img_01441.jpg
img_01337.jpg
img_01085.jpg
img_00233.jpg
img_00823.jpg
img_01412.jpg
img_00009.jpg
img_01175.jpg
img_00912.jpg
img_01503.jpg
img_00308.jpg
img_01709.jpg
img_00932.jpg
img_01563.jpg
img_00741.jpg
img_01029.jpg
img_01518.jpg
img_00846.jpg
img_00479.jpg
img_00367.jpg
img_01474.jpg
img_01549.jpg
img_00471.jpg
img_00544.jpg
img_00635.jpg
img_00560.jpg
img_00290.jpg
img_00160.jpg
img_01540.jpg
img_00084.jpg
img_00123.jpg
img_00180.jpg
img_01132.jpg
img_00613.jpg
img_00833.jpg
img_01065.jpg
img_00222.jpg
img_00355.jpg
img_01201.jpg
img_00289.jpg
img_01280.jpg
img_01300.jpg
img_01126.jpg
img_01204.jpg
img_00162.jpg
img_00044.jpg
img_00595.jpg
img_00727.jpg
img_01463.jpg
img_00244.jpg
img_00557.jpg
img_00047.jpg
img_00306.jpg
img_00209.jpg
img_00641.jpg
img_01695.jpg
img_00018.jpg
img_01528.jpg
img_01393.jpg
img_01372.jpg
img_00714.jpg
img_01301.jpg
img_01214.jpg
img_00489.jpg
img_01696.jpg
img_00184.jpg
img_01273.jpg
img_00785.jpg
img_01239.jpg
img_01179.jpg
img_01284.jpg
img_00

In [ ]:
np.save(path+'/save/test_des_%dx%d_%d.npy'%(img_size,img_size,jump),test_des)

In [58]:
np.save(path+'/save/test_file_name.npy',test_file_name)

In [55]:
test_des = np.load(path+'/save/test_des_288x288_6.npy')

In [ ]:
test_file_name = np.load(path+'/test_file_name.npy')

In [73]:
test_histogram = SPM_2(test_des,all_codebook,codebook_len,img_size,jump)
test_histogram_HI = HistogramIntersection(test_histogram,all_train_histogram)






  0%|          | 0/1712 [00:00<?, ?it/s]/home/jslee/.conda/envs/bow/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
/home/jslee/.conda/envs/bow/lib/python3.6/site-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
/home/jslee/.conda/envs/bow/lib/python3.6/site-packages/ipykernel_launcher.py:29: VisibleDeprec

0 %
0 %
0 %
0 %







  0%|          | 6/1712 [00:00<01:40, 17.03it/s]




  0%|          | 8/1712 [00:00<01:38, 17.35it/s]

0 %
0 %
0 %
0 %







  1%|          | 10/1712 [00:00<01:37, 17.43it/s]




  1%|          | 12/1712 [00:00<01:35, 17.74it/s]

...






100%|█████████▉| 1705/1712 [01:47<00:00, 16.71it/s]




100%|█████████▉| 1707/1712 [01:47<00:00, 16.50it/s]

100 %
100 %
100 %
100 %







100%|█████████▉| 1709/1712 [01:47<00:00, 16.61it/s]




100%|██████████| 1712/1712 [01:47<00:00, 15.87it/s]

100 %
100 %
100 %
100 %








  0%|          | 0/1712 [00:00<?, ?it/s]





  0%|          | 0/3060 [00:00<?, ?it/s]





100%|██████████| 3060/3060 [00:00<00:00, 16281.88it/s]





  0%|          | 1/1712 [00:00<05:26,  5.24it/s]





  0%|          | 0/3060 [00:00<?, ?it/s]





100%|██████████| 3060/3060 [00:00<00:00, 16488.72it/s]





  0%|          | 2/1712 [00:00<05:26,  5.24it/s]





  0%|          | 0/3060 [00:00<?, ?it/s]





100%|██████████| 3060/3060 [00:00<00:00, 16373.49it/s]





  0%|          | 3/1712 [00:00<05:26,  5.23it/s]





  0%|          | 0/3060 [00:00<?, ?it/s]





100%|██████████| 3060/3060 [00:00<00:00, 16265.50it/s]





  0%|          | 4/1712 [00:00<05:27,  5.21it/s]





  0%|          | 0/3060 [00:00<?, ?it/s]





100%|██████████| 3060/3060 [00:00<00:00, 15575.93it/s]





  0%|          | 5/1712 [00:00<05:31,  5.14it/s]





  0%|          | 0/3060 [00:00<?, ?it/s]





100%|██████████| 3060/3060 [00:00<00:00, 16418.29it/s]





  0%|          | 6/1712 [00:01<05:29,  5.1

In [74]:
all_predict = model_all.predict(test_histogram_HI)

In [75]:
## 제출형식을 불러와 사전형으로 저장
submit_form = np.array(pd.read_csv(path + '/submit.csv'))

submit_dict = {}
for x in range(len(submit_form)):
    submit_dict[submit_form[x][0]] = submit_form[x][1]

In [81]:
all_predict.shape

(1712,)

In [84]:
print(test_file_name)

['img_01441.jpg', 'img_01337.jpg', 'img_01085.jpg', 'img_00233.jpg', 'img_00823.jpg', 'img_01412.jpg', 'img_00009.jpg', 'img_01175.jpg', 'img_00912.jpg', 'img_01503.jpg', 'img_00308.jpg', 'img_01709.jpg', 'img_00932.jpg', 'img_01563.jpg', 'img_00741.jpg', 'img_01029.jpg', 'img_01518.jpg', 'img_00846.jpg', 'img_00479.jpg', 'img_00367.jpg', 'img_01474.jpg', 'img_01549.jpg', 'img_00471.jpg', 'img_00544.jpg', 'img_00635.jpg', 'img_00560.jpg', 'img_00290.jpg', 'img_00160.jpg', 'img_01540.jpg', 'img_00084.jpg', 'img_00123.jpg', 'img_00180.jpg', 'img_01132.jpg', 'img_00613.jpg', 'img_00833.jpg', 'img_01065.jpg', 'img_00222.jpg', 'img_00355.jpg', 'img_01201.jpg', 'img_00289.jpg', 'img_01280.jpg', 'img_01300.jpg', 'img_01126.jpg', 'img_01204.jpg', 'img_00162.jpg', 'img_00044.jpg', 'img_00595.jpg', 'img_00727.jpg', 'img_01463.jpg', 'img_00244.jpg', 'img_00557.jpg', 'img_00047.jpg', 'img_00306.jpg', 'img_00209.jpg', 'img_00641.jpg', 'img_01695.jpg', 'img_00018.jpg', 'img_01528.jpg', 'img_01393.jp

In [79]:
## 제출형식의 이미지 이름에 해당하는 테스트 이미지의 답을 대입

for x in tqdm(range(len(test_file_name))):
    submit_dict[test_file_name[x]] = all_predict[x]







  0%|          | 0/1713 [00:00<?, ?it/s]


IndexError: index 1712 is out of bounds for axis 0 with size 1712